In [1]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from collections import defaultdict

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sys import getsizeof
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/felicityzhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/felicityzhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
inverted_index = defaultdict(dict)
STOP_WORDS = set(stopwords.words('english'))

In [4]:
def lemma(word):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)

In [5]:
def remove_u(word):
    word_u = (word.encode('unicode-escape')).decode("utf-8", "strict")
    if r'\u' in word_u: 
        return word_u.split('\\u')[1]
    return word

In [6]:
#Read json file
news_json = open("title_links.json")
news = json.load(news_json)
for k, v in news.items():
    url = "https://www.ucihealth.org" + v
    html = urlopen(url)
    soup = BeautifulSoup(html)
    k = remove_u(k)
    print(k)
    
    #Fetch Contents
    contents = soup.find_all('p')
    article = []
    for c in contents:
        elem = c.get_text(strip=True)
        elem = elem.replace('\xa0', ' ')
        if elem != '':
            article.append(elem)
    
    #all_text to lower case, split by nonalphanumeric characters
    for text in article:
        text = re.sub(r'[^a-z0-9]', ' ', text.lower())
        
        for word in text.split():
            if len(word) > 2:
                word=lemma(word)
                if word not in STOP_WORDS:
                    if word not in inverted_index:
                        word_dict = {k:1}
                        inverted_index[word] = word_dict
                    else:
                        if k in inverted_index[word]:
                            inverted_index[word][k] += 1
                        else:
                            word_dict = {k:1}
                            inverted_index[word].update(word_dict)



UCI Health analysis shows increase in penetrating trauma related to COVID-19 stay-at-home changes
2019s Best Maternity Care Hospitals
UCI Medical Center is the first hospital on the West Coast to use portable MRI technology
UCI to build world-class hospital on Irvine campus
UCI campus COVID-19 patient vaccination site opens
UCI team at work on preemptive vaccine for all coronaviruses
Raising the bar on excellence
Anaheim fire paramedic who survives COVID-19 warns against New Years' parties
UCI Health sets up field hospital to handle surge of patients
The Leapfrog Group gives UCI Health 13th "A" grade in patient safety
UCI Health receives COVID-19 vaccine
Can children play sports safely during a pandemic?
2018Giant of Cancer Care
Antibody screening shows COVID-19 nearly 7 times more prevalent in O.C. than expected
UCI Anti-Cancer Challenge participants fund 15 grants
2014 Newport Beach offers science-based integrative health
New UCI Health advanced care center to offer greater access to

In [ ]:
#sort frequencies for each key words
final_dict = defaultdict(list)
for k,v in inverted_index.items():
    sorted_tuples = sorted(v.items(), key=lambda item: -item[1])
    final_dict[k] = sorted_tuples
    

In [ ]:
with open("inverted_index_words.json", "w") as outfile:  
    json.dump(final_dict, outfile)